# Intro to CasADi

## Goal
* Introduce the symbolic framework and syntax for the CasADi library through implementation, especially in comparison to numpy.
* Go through the topics introduced in [CasADi Docs](https://web.casadi.org/docs/) (which is written more like a tutorial rather than code documentation), skipping any topics that are directly not relevant to trajectory optimization via direct collocation.
* Introduce an example of a Nonlinear Program (NLP) solved through the CasADi opti stack, and comparing it with with the "raw" NLP interface for CasADi.
* It is still recommended that you read the references, at least Sec. 3 and 4 of CasADi Docs to get a better description of each data structure. The purpose of this notebook is to supplement it through interactive implementations and to offer my own practical advice using this library, to hopefully accelerate the learning process for someone with numpy experience.

## References
* \[1\] [CasADi Docs](https://web.casadi.org/docs/) - The basic page from which most of this notebook is designed off of.
* \[2\] [CasADi Blog Post Detailing Opti](https://web.casadi.org/blog/opti/) - Opti is a relatively new functionality added to CasADi, which is explained in more detail here than the main Docs page.
* \[3\] [CasADi Python API Docs](https://web.casadi.org/python-api/) - Detailed docs page for the CasADi interface for python (a MATLAB interface also exists).

In [ ]:
# Import libraries. Install them through pip, conda, etc as necessary
import numpy as np
import casadi as ca

## Symbolic Framework (Sec. 3)

CasADi introduces it's own symbolic framework built on custom data structures with native support for key functionalities such as very high quality implementations of autogradients, matrix arithmetic, and high compatibility with numpy. All three of these functionalities were missing from the library I experimented with previously to switching to CasADi (The [Pyomo library](https://pyomo.readthedocs.io/en/stable/)), which I why I believe that CasADi is a good tool for trajectory optimization through python.

### SX Class (Sec. 3.1)

In practical terms, the SX class is a type of symbolic matrix class where each element of the matrix is its own scalar symbolic variable (see \[1\] for a more formal discussion).

It is defined through putting its name inside quotation marks (I'm not quite sure what this does. Presumably this is the actual variable name that's passed to the NLP solver, and not the python variable name), and specifying its dimensions:

In [ ]:
x = ca.SX.sym('x')
y = ca.SX.sym('y', 5)
z = ca.SX.sym('z', 4, 2)
v = ca.SX.sym('v', 2, 3, 4) # note that I never use this, since casadi matrices are meant to be used as 2D matrices

print(x)
print(y)
print(z)
print(v)

Notice how whenever the variable is a matrix, it automatically fills the matrix with scalar symbolic variables corrsponding to each element of the matrix.

Also note the ordering of elements, which is **not** the same as numpy's ordering of matrix elements. This can create bugs, for example when comparing the output of reshape() functions between casadi and numpy objects:

In [ ]:
z_np = np.array([[0, 1], [2, 3], [4, 5], [6, 7]])
print('\nCasadi is ordered row first:\n', z)
print('\nNumpy is ordered column first:\n', z_np)

print('\nThis ordering is reflected in how the reshape function orders elements:')
print(np.reshape(z_np, (1, 8)))
print(ca.reshape(z, 1, 8))

print('\nBut numpy reshape can be made to order elements in the same way as casadi:\n')
print(np.reshape(z_np, (1, 8), order='F'))

You can construct SX matrices with functions similar to numpy functions. Some unique features here are the distinction between structural vs actual zeros and sparse vs dense matrices, which presumably makes a big difference to NLP solution performance. See \[1\] for details:

In [ ]:
print('\nArbitrary symbolic matrix:\n', ca.SX.sym('x',2,3))
print('\nDense matrix of actual zeros:\n', ca.SX.zeros(2,3))
print('\nSparse matrix of structural zeros:\n', ca.SX(2,3))
print('\nSparse identity matrix:\n', ca.SX.eye(3))
z_np = np.array([[0, 1], [2, 3], [4, 5], [6, 7]])
print('\nMatrix from numpy array:\n', ca.SX(z_np))

### DM Class (Sec. 3.2)

In practical terms, the DM class is a matrix meant to hold numerical values, which I typically encounter as an output of a casadi function. I always convert it to a numpy array afterwards, since numpy arrays are generally easier to work with (eg. a lot more operations are supported with it).

In [ ]:
x = ca.SX.sym('x') # define a symbolic variable
f = ca.Function('f', [x], [x**2+10]) # define a function. This will be elaborated later.

print('\nf(3):', f(3))
print('\ntype(f(3)): ', type(f(3)))
print('\nf(3).shape: ', f(3).shape)
print('\nValue converted to numpy array: ', np.array(f(3)))

### MX Class (Sec. 3.3)

In practical terms, the MX class is the "true" matrix class of casadi. Operations on MX matrices are not limited to scalar operations performed element-wise, offering more generality. This is the main casadi datatype that I use since many casadi functions only accept MX as input. Unfortunately I also find that it is also much harder to debug expressions using MX, since I can't see what operations are being performed at an element level (see \[1\] for a more formal discussion).

MX matrices are initialized similarly to SX matrices:

In [ ]:
x = ca.MX.sym('x',2,2)
y = ca.MX.sym('y')

print(x)
print(y)

Note above that we are not shown what the elements inside x and y are, in contrast to if we defined similar matrices using SX:

In [ ]:
x_SX = ca.SX.sym('x_SX',2,2)
y_SX = ca.SX.sym('y_SX')

print(x_SX)
print(y_SX)

Another difference between SX and MX can be seen when performing arithmetic operations:

In [ ]:
print('\nArithmetic operations on MX are represented as \"true\" matrix arithmetic:\n', x + y)
print('\nArithmetic operations on SX are represented as a matrix of scalar arithmetic\n', x_SX + y_SX)

Defining operations using MX is presumably far more efficient for the algorithmic differentiation, but it also makes it much harder for humans to read, complicating the debugging process:

In [ ]:
# create an MX matrix, populate it with some values
A = ca.MX.sym('A',2,2)
A[0,0] = x[0]
A[0,1] = x[1]
A[1,0] = 2
A[1,1] = y

print('\nCan you tell what A is from looking at this printout? I can\'t:\n', A)

# create an SX matrix, populate it using similar method
A_SX = ca.SX.sym('A_SX',2,2)
A_SX[0,0] = x_SX[0]
A_SX[0,1] = x_SX[1]
A_SX[1,0] = 2
A_SX[1,1] = y_SX

print('\nMuch easier to understand:', A_SX)

As a final note, in general MX and SX classes cannot be mixed together in operations. This discussed in more detail in Sec. 3.4 in \[1\]. Discussion about conversion between the classes is given in Sec. 4.2 in \[1\].

### Getting and Setting Elements (Sec. 3.5.1)

Getting and setting elements for casadi matrices are syntactically similar to numpy arrays, however some functional differences do occur. This is likely due to the primary interpretation of casadi matrices being of 2D matrices, being more similar to MATLAB matrices rather than numpy arrays.

We can do a side to side comparison of getting elements with casadi matrices and numpy arrays:

In [ ]:
# demo getting elements for casadi matrices
M = ca.SX([[3,7],[4,5]])
print('\nM:', M)
print('\nM[0,0]:', M[0,0])
print('\nM[1,0]:', M[1,0])
print('\nM[1,:]:', M[1,:])
print('\nM[1,:].shape:', M[1,:].shape)
print('\nM[1,:][0]:', M[1,:][0])

In [ ]:
# compare with getting elements for numpy array
M_np = np.array([[3,7],[4,5]])
print('\nM_np:\n', M_np)
print('\nM_np[0,0]:', M_np[0,0])
print('\nM_np[1,0]:', M_np[1,0])
print('\nM_np[1,:]:', M_np[1,:])
print('\nM_np[1,:].shape:', M_np[1,:].shape)
print('\nM_np[1,:][0]:', M_np[1,:][0])

Note in the above that the slice of a casadi matrix is still a 2D matrix, with "1" being one of its two dimensions. In contrast, a slice of a numpy array is a one dimensional vector. Again, this is likely due to casadi matrices being designed more similarly to MATLAB matrices rather than numpy arrays.

Additionally, getting an element using one index has very different behavior for casadi matrices and numpy arrays:

In [ ]:
M = ca.SX([[3,7],[4,5]])
print('\nM:', M)
print('\nM[0]:', M[0])
print('\nM[1]:', M[1])

In [ ]:
M_np = np.array([[3,7],[4,5]])
print('\nM_np:\n', M_np)
print('\nM_np[0]:', M_np[0])
print('\nM_np[1]:', M_np[1])

In the above, we see that using one index to reference an element from a casadi matrix references a single element starting from the upper left corner and column-wise to the bottom right corner (the same matrix element order discussed previously in the SX class section). This is different from numpy, where a single element acccess gets the first element of the list of lists. Again, these functionalities reflect the respective interpretations of the datatypes (2D matrices vs arbitrarily dimensioned containers).

Finally, we note that casadi does some odd things with pass by reference vs value with matrix referencing:

In [ ]:
M = ca.SX([[3,7],[4,5]])
print('\nM:', M)
M[0,0] = 1
print('\nM after running M[0,0] = 1:', M)
M[1,:] = 2
print('\nM after running M[1,:] = 2:\n', M)

M = ca.SX([[3,7],[4,5]])
print('\nM after resetting:', M)
M[0,:][0,0] = 1
print('\nM after running M[0,:][0,0] = 1:', M)

In the last case above, we see that M is unchanged. From \[1\], "Unlike Python’s NumPy, CasADi slices are not views into the data of the left hand side; rather, a slice access copies the data". As with before, I would recommend reading through Sec. 3.5.1 of \[1\] to get a detailed discussion of the features discussed so far, and also to see what other indexing functionality is offered by casadi.

### Operations on Casadi Matrices (Sec. 3.6 - 3.8)

Basically, most operations that you would want to use on casadi matrices (ie operations that you can perform on numpy arrays) are implemented. The only thing to note is to make sure to use casadi's implementation of functions and not numpy's, since all of these operations must be kept track of by the autogradient framework.
* (the necessity to distinguish between numpy and casadi implementations is why I chose to run `import casadi as ca` and not `from casadi import *` as in what \[1\] does)
* (casadi disallows you from using a numpy function on a casadi object by throwing an error)

In [ ]:
A = ca.SX.sym('A', 2, 2)
print('\nA:', A)
print('\nA.shape:\n', A.shape)
print('\nca.cos(A):', ca.cos(A))
print('\nA*A:', A*A)
print('\nA @ A:', A@A)
print('\nA.T:', A.T)

b = ca.SX.sym('b', 2)
print('\nb: \n', b)
print('\nA\\b:\n', ca.solve(A,b))
print('\nA + b:', A+b)

Note that the last operation involves something similar to numpy's array broadcasting. However, not all of the features of array broadcasting translate directly to casadi arrays, so I sometimes find myself having to write loops where a similar operation in numpy could've been written in one line with array broadcasting (TODO: add an example of this behavior).

Occasionally a function has a different name than its numpy counterpart, so I need to look for it in the casadi python API page \[3\]. Also note that this page has very little documentation in terms of what each function does, which can make it challenging to know what it does, and also to find it through `ctrl-F`.

## Functions (Sec. 4)

The ability to define functions significantly streamlines the process of writing optimization problems, especially for robotics applications where similar computations need to be called many times. Casadi elegantly handles autodifferentiation, so that casadi function objects store what the function does in addition to all of the necessary information about its gradients so that you never have to worry about it.

Defining functions is very simple:

In [ ]:
# define input variables used to define the function
x = ca.SX.sym('x',2)
y = ca.SX.sym('y')

# define the function
f = ca.Function('f',[x,y], [ca.sin(y)*x])
print(f)

# my functions are typically more complex, so I usually define a variable for the output first
z = ca.sin(y)*x
f2 = ca.Function('f2',[x,y], [z])
print(f2)

Function calls work exactly as you would expect it to. You can call it on symbolic or numeric inputs.

In [ ]:
# test function on symbolic SX input
x_test = ca.SX.sym('x_test',2)
y_test = ca.SX.sym('y_test')
print('\nsymbolic SX input:\n', f(x_test, y_test))

# test function on numerical input
x_test_np = np.array([3, 4])
y_test_np = 5
print('\nnumeric input:\n', f(x_test_np, y_test_np))

# test function on symbolic MX input
x_test_MX = ca.MX.sym('x_test_MX',2)
y_test_MX = ca.MX.sym('y_test_MX')
print('\nsymbolic MX input:\n', f(x_test_MX, y_test_MX))

Although we haven't discussed how to use autodifferentiation (because we don't need to explicitly-the NLP solver classes handle this automatically), we can see that casadi function objects natively support autogradients:

In [ ]:
print('f.jacobian():', f.jacobian())

## Nonlinear Programming with CasADi (Sec. 4.5 and [2])

Now that we have familiarity with the main classes that make up casadi, we are ready to implement NLPs with it. Casadi supports two methods of defining and solving NLPs, where my recommended method is the more recent `opti` stack. We will discuss this one first, then compare it with the "raw" NLP definition method built into casadi.

Consider the following NLP:
\begin{align}
\min_{x,y} & (y-x^2)^2 \\
\text{ s.t. } & x^2+y^2=1 \\
& x + y \geq 1
\end{align}
We begin by constructing an opti object:

In [ ]:
opti = ca.Opti()

Then we define decision variables for the NLP. The argument is the dimension of the decision variable, which can be left empty for scalar variables:

In [ ]:
x = opti.variable()
y = opti.variable()

Now we define the optimization objective.

In [ ]:
opti.minimize( (y-x**2)**2 )

Next, we define constraints:

In [ ]:
opti.subject_to( x**2+y**2 == 1 )
opti.subject_to( x+y >= 1)

Finally, we specify [IPOPT](https://github.com/coin-or/Ipopt ) as the optimization solver library to use, and solve it. IPOPT is well known as a fairly robust open source solver for NLPs, but other options exist for QPs or LPs. A basic binary installation of IPOPT comes bundled with casadi, but it's installation can likely be optimized for faster solves (eg. the IPOPT installation that came with my casadi installation with PIP uses the MUMPS linear system solver, which can be replaced with Intel's MKL to supposedly speed up solves significantly. I haven't experimented with this myself).

In [ ]:
opti.solver('ipopt')
sol = opti.solve()

The solution can then be extracted.

In [ ]:
x_sol = sol.value(x)
y_sol = sol.value(y)

print('\nNLP solution for x:', x_sol)
print('\nNLP solution for y:', y_sol)

print('\ntype(x_sol):', type(x_sol))

Next, we can explore additional options that we have with solving NLPs. Consider the following optimization problem.
\begin{align}
\min_{x_1,x_2} & (x_2-x_1^2)^2 \\
\text{ s.t. } & 2 \leq x_1^2+x_2^2 \leq 3 \\
& x + y \geq p
\end{align}
where $p$ is a parameter value whos numerical value can be set after defining the mathematical structure of the NLP (eg. for an MPC, the NLP structure can be defined once in a constructor and initial conditions will be supplied at every call to the MPC).

The above NLP can be defined in code as follows.

In [ ]:
opti = ca.Opti()

# decision variables
x = opti.variable(2) # define x as a vector (or technically a 2x1 matrix)
p = opti.parameter() # define NLP parameters

# objective
opti.minimize( (x[1]-x[0]**2)**2 )

# constraints. Note that opti.bounded removes the need to use two inequalities
opti.subject_to(opti.bounded(2, ca.sum1(x**2), 3))
opti.subject_to( ca.sum1(x) >= p)

Some observations before moving one with the solution. Note that when we define decision variables or parameters as matrices, the datatype is automatically a casadi MX type. Also, a 1D vector is implemented as a 2D matrix with one of its dimensions being "1" (recall that casadi matrices are meant to be 2D matrices).

In [ ]:
print('type(x):', type(x))
print('x.shape:', x.shape)

This time, we also explore some additional options we have with solving NLPs:
1. Setting the value of parameters
1. Adjusting IPOPT settings, such as output verbosity. See [IPOPT documentation](https://coin-or.github.io/Ipopt/OPTIONS.html) for the full list of options
1. Setting initial guesses for optimization. This is crucial for highly nonlinear and high dimensional NLPs (eg. robotics problems)

Try experimenting with some of these values here:

In [ ]:
# set parameter value
# try experimenting with values such as 0, 2, 3
opti.set_value(p, 0)

# toggle IPOPT options
p_opts = {}
#p_opts = {'print_time' : False} # try using this line instead, combined with print_level = 0 for s_opts

s_opts = {'print_level': 5} # try experimenting with values between 0 and 12 inclusive. Default is 5
#s_opts = {"max_iter": 3} # try using this line instead

# set initial guess
# try using the second line instead, and notice how the solution changes depending on the initial guess
x_guess = np.array([0, 0])
#x_guess = np.array([-100, 100])
opti.set_initial(x, x_guess)

opti.solver('ipopt', p_opts, s_opts)
sol = opti.solve()
x_sol = sol.value(x)
print('\nNLP solution for x:', x_sol)

Finally, we note that an alternative method of defining NLPs exist in casadi, which is the method used in some older examples (eg. the [casadi example pack supplied by casadi](https://web.casadi.org/get/) (`ctrl-f` on the link to find the example pack link)). I will show you how to define the same NLP as before using this method and hopefully convince you that opti is easier to work with.

We first note that the old method has to be defined with an NLP of the form:

\begin{align}
\min_{x} & f(x,p) & \\
\text{ s.t. } & x_{lb} \leq x \leq x_{ub}\\
& g_{lb} \leq g(x,p) \leq g_{ub}
\end{align}

where $x$ is the decision variable vector, and $p$ is the parameter vector. Despite this standard form being able to represent any arbitrary NLP, the need to explicitly convert your problem into this form is an extra step that can potentially introduce bugs in your calculation.

Recalling that the NLP considered before is given by the form
\begin{align}
\min_{x_1,x_2} & (x_2-x_1^2)^2 \\
\text{ s.t. } & 2 \leq x_1^2+x_2^2 \leq 3 \\
& x + y \geq p
\end{align}
the corresponding standard form NLP is given by
\begin{align}
f(x,p) & = (x_2-x_1^2)^2 \\
g(x,p) & =
\begin{bmatrix}
x_1^2+x_2^2\\
x_1+x_2-p
\end{bmatrix} \\
g_{lb} = \begin{bmatrix}
2\\
0
\end{bmatrix}\text{, }
& g_{ub} = \begin{bmatrix}
3\\
\infty
\end{bmatrix}
\end{align}
where there are no bounds on $x$.

Implementing this in code looks like the following (using a coding style taken from the vdp_collocation example from the casadi example pack, which is optimized for scalability rather than simplicity):

In [ ]:
# define decision variable and parameters. They must be 1D vectors
x = ca.MX.sym('x', 2)
p = ca.MX.sym("p")

# define objective
f = (x[1]-x[0]**2)**2

g = [] # container to hold constraint function g(x,p)
lbg = [] # container to hold lower bound for g(x,p)
ubg = [] # container to hold upper bound for g(x,p)
arg = {} # container to hold optimization arguments

# first constraint
g.append(ca.sum1(x**2))
lbg.append(np.array([2]))
ubg.append(np.array([3]))

# second constraint
g.append(ca.sum1(x) - p)
lbg.append(np.array([0]))
ubg.append(np.array([np.inf]))

# intial guess
x_guess = np.array([0, 0])

arg['lbg'] = np.concatenate(lbg) #lower bound constraints
arg['ubg'] = np.concatenate(ubg) #lower bound constraints
arg['x0'] = x_guess #initial guess
arg['p']  = 0 # numerial value for parameter

g = ca.vertcat(*g)
nlp = {'x':x, 'f':f, 'g':g, 'p': p}
S = ca.nlpsol("S", "ipopt", nlp)
result = S(**arg)

x_sol = result['x']
print('\nNLP solution for x:', x_sol)

Hopefully that gives you an idea of why I love the opti stack so much. In particular, although it wasn't shown in this simple example, the need to have the decision variable be a single vector and not a series of separate matrix variables significantly complicates NLP definition for robotic trajectory optimization applications.

For example, if you are defining a trajectory optimization problem for a robot, it's natural to define your decision variable as a matrix of robot states at each timestep, with each column corresponding to a state at a given timestep. Additionally, if you have any other decision variables like joint contact states, then you need to concatenate this to the other decision variables, resulting in a decision variable vector that has different physical interpretations depending on which portion of the variable you are referring to.